# **Capstone Project**
# **The Battle of Neighborhoods in Cologne: Bars**

### This is a final project part of the IBM Data Science Professional Certificate, in which we are going to explore the data of the emblematic city of Cologne, Germany and specially the bars of the zone.

### I have great interest in exploring the bars because Cologne is home of the famous Kölsh beer which is by most the best beer I have drunk and it would be great that you could know where are the best bars to try this local specialty.

## 1. Introduction

### This project is intented to give an overview to the tourists and the students in the area of which are the best bars they could go in order to finish in the best way their visit to Cologne, one of the main things that someone needs to try here is a Kölsch beer, so it would be exceptional if the could enjoy it in the best place, so this tool will provide support to the different needs of the people in terms of having a great Bier!!!

## 2. Data Preparation

### First we import all the necessary packages and libraries in order to work, this will help us in the second part of the project in order to cluster the data

In [1]:
import numpy as np # library to handle data in a vectorized manner
import requests

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### It is important to gather the neighboorhoos and boroughs information of Cologne, in order to do that we extract it from Wikipedia, but we will need to clean-up this data to make it work

In [7]:
url = 'https://en.wikipedia.org/wiki/Districts_of_Cologne'
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[1]
df.head()

,Map,Coat,City district,City parts,Area,Population1,Pop. density,District Councils,Town Hall
0,NaN,NaN,District 1 Köln-Innenstadt,"Altstadt-Nord, Altstadt-Süd, Deutz, Neustadt-N...",16.4 km²,127.033,7.746/km²,"Bezirksksamt Innenstadt Brückenstraße 19, D-50...",NaN
1,NaN,NaN,District 2 Köln-Rodenkirchen,"Bayenthal, Godorf, Hahnwald, Immendorf, Marien...",54.6 km²,100.936,1.850/km²,"Bezirksamt Rodenkirchen Hauptstraße 85, D-5099...",NaN
2,NaN,NaN,District 3 Köln-Lindenthal,"Braunsfeld, Junkersdorf, Klettenberg, Lindenth...",41.6 km²,137.552,3.308/km²,"Bezirksamt Lindenthal Aachener Straße 220, 509...",NaN
3,NaN,NaN,District 4 Köln-Ehrenfeld,"Bickendorf, Bocklemünd/Mengenich, Ehrenfeld, N...",23.8 km²,103.621,4.348/km²,"Bezirksamt Ehrenfeld Venloer Straße 419 – 421,...",NaN
4,NaN,NaN,District 5 Köln-Nippes,"Bilderstöckchen, Longerich, Mauenheim, Niehl, ...",31.8 km²,110.092,3.462/km²,"Bezirksamt NippesNeusser Straße 450,D-50733 Köln",NaN


### Since we only need the information of the Boroughs and Neighborhoods we only extract this information

In [8]:
df = df[['City district','City parts']]
df.head()

,City district,City parts
0,District 1 Köln-Innenstadt,"Altstadt-Nord, Altstadt-Süd, Deutz, Neustadt-N..."
1,District 2 Köln-Rodenkirchen,"Bayenthal, Godorf, Hahnwald, Immendorf, Marien..."
2,District 3 Köln-Lindenthal,"Braunsfeld, Junkersdorf, Klettenberg, Lindenth..."
3,District 4 Köln-Ehrenfeld,"Bickendorf, Bocklemünd/Mengenich, Ehrenfeld, N..."
4,District 5 Köln-Nippes,"Bilderstöckchen, Longerich, Mauenheim, Niehl, ..."


### As you can see the data set contains some information within the cells that we don´t need, so we will apply some steps to extract the extract and separate the information, so that later we can find the latitudes and longitudes

In [9]:
df.rename(columns={'City district': 'BoroughPrev', 'City parts': 'Neighborhoods'}, inplace=True)
df.head()

C:\Users\dante\anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,BoroughPrev,Neighborhoods
0,District 1 Köln-Innenstadt,"Altstadt-Nord, Altstadt-Süd, Deutz, Neustadt-N..."
1,District 2 Köln-Rodenkirchen,"Bayenthal, Godorf, Hahnwald, Immendorf, Marien..."
2,District 3 Köln-Lindenthal,"Braunsfeld, Junkersdorf, Klettenberg, Lindenth..."
3,District 4 Köln-Ehrenfeld,"Bickendorf, Bocklemünd/Mengenich, Ehrenfeld, N..."
4,District 5 Köln-Nippes,"Bilderstöckchen, Longerich, Mauenheim, Niehl, ..."


In [10]:
df[['BoroughLast','Borough']] = df.BoroughPrev.str.split("-",expand=True,)
df.head()

C:\Users\dante\anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,BoroughPrev,Neighborhoods,BoroughLast,Borough
0,District 1 Köln-Innenstadt,"Altstadt-Nord, Altstadt-Süd, Deutz, Neustadt-N...",District 1 Köln,Innenstadt
1,District 2 Köln-Rodenkirchen,"Bayenthal, Godorf, Hahnwald, Immendorf, Marien...",District 2 Köln,Rodenkirchen
2,District 3 Köln-Lindenthal,"Braunsfeld, Junkersdorf, Klettenberg, Lindenth...",District 3 Köln,Lindenthal
3,District 4 Köln-Ehrenfeld,"Bickendorf, Bocklemünd/Mengenich, Ehrenfeld, N...",District 4 Köln,Ehrenfeld
4,District 5 Köln-Nippes,"Bilderstöckchen, Longerich, Mauenheim, Niehl, ...",District 5 Köln,Nippes


In [11]:
df = df[['Borough','Neighborhoods']]
df.drop([9,10], axis=0, inplace= True)
df

,Borough,Neighborhoods
0,Innenstadt,"Altstadt-Nord, Altstadt-Süd, Deutz, Neustadt-N..."
1,Rodenkirchen,"Bayenthal, Godorf, Hahnwald, Immendorf, Marien..."
2,Lindenthal,"Braunsfeld, Junkersdorf, Klettenberg, Lindenth..."
3,Ehrenfeld,"Bickendorf, Bocklemünd/Mengenich, Ehrenfeld, N..."
4,Nippes,"Bilderstöckchen, Longerich, Mauenheim, Niehl, ..."
5,Chorweiler,"Blumenberg, Chorweiler, Esch/Auweiler, Fühling..."
6,Porz,"Eil, Elsdorf, Ensen, Finkenberg, Gremberghoven..."
7,Kalk,"Brück, Höhenberg, Humboldt/Gremberg, Kalk, Mer..."
8,Mülheim,"Buchforst, Buchheim, Dellbrück, Dünnwald, Flit..."


### The final step of extracting the necessary information is by splitting the information of every cell into different rows, we can do this because every neighborhood is separated by a comma.

In [14]:
df = pd.DataFrame(df.Neighborhoods.str.split(',').tolist(), index=df.Borough).stack()
df = df.reset_index([0, 'Borough'])
df.columns = ['Borough', 'Neighborhoods']
df.head(10)

,Borough,Neighborhoods
0,Innenstadt,Altstadt-Nord
1,Innenstadt,Altstadt-Süd
2,Innenstadt,Deutz
3,Innenstadt,Neustadt-Nord
4,Innenstadt,Neustadt-Süd
5,Rodenkirchen,Bayenthal
6,Rodenkirchen,Godorf
7,Rodenkirchen,Hahnwald
8,Rodenkirchen,Immendorf
9,Rodenkirchen,Marienburg


### In order to get the latitude and longitude of each neighborhood we use geopy client, joining it with the set of the boroughs and neighborhoods our data frame is completed

In [15]:
cologne_coor = pd.DataFrame(columns = ['Latitude','Longitude'])
geolocator = Nominatim(user_agent="Cologne")
for row in df['Neighborhoods']:
    location = geolocator.geocode(row)
    cologne_coor = cologne_coor.append({'Latitude':location.latitude, 'Longitude':location.longitude}, ignore_index=True)
df = df.join(cologne_coor)
df

,Borough,Neighborhoods,Latitude,Longitude
0,Innenstadt,Altstadt-Nord,50.942265,6.953188
1,Innenstadt,Altstadt-Süd,50.929181,6.955329
2,Innenstadt,Deutz,50.936759,6.973046
3,Innenstadt,Neustadt-Nord,50.947906,6.941059
4,Innenstadt,Neustadt-Süd,50.926560,6.940406
5,Rodenkirchen,Bayenthal,50.911058,6.969238
6,Rodenkirchen,Godorf,50.849154,6.975350
7,Rodenkirchen,Hahnwald,49.662407,9.265221
8,Rodenkirchen,Immendorf,48.646985,16.127598
9,Rodenkirchen,Marienburg,54.035936,19.034882


### Finally in order to verify our data we map the latitude and longitude of each neighborhood on the map of Cologne

In [16]:
address = 'Cologne'

geolocator = Nominatim(user_agent="Cologne")
location = geolocator.geocode(address)
latitude_cologne = location.latitude
longitude_cologne = location.longitude
map_cologne = folium.Map(location=[latitude_cologne, longitude_cologne], zoom_start=12)

# add markers to map
for lat, lng, postal in zip(df['Latitude'],df['Longitude'],df['Neighborhoods']):
    label = str(postal)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lng)],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cologne)  
    
map_cologne